# Intro

[PyTorch](https://pytorch.org/) is a very powerful machine learning framework. Central to PyTorch are [tensors](https://pytorch.org/docs/stable/tensors.html), a generalization of matrices to higher ranks. One intuitive example of a tensor is an image with three color channels: A 3-channel (red, green, blue) image which is 64 pixels wide and 64 pixels tall is a $3\times64\times64$ tensor. You can access the PyTorch framework by writing `import torch` near the top of your code, along with all of your other import statements.

This guide will help introduce you to the functionality of PyTorch, but don't worry too much about memorizing it: the assignments will link to relevant documentation where necessary.

In [79]:
import torch

# Why PyTorch?

One important question worth asking is, why is PyTorch being used for this course? There is a great breakdown by [the Gradient](https://thegradient.pub/state-of-ml-frameworks-2019-pytorch-dominates-research-tensorflow-dominates-industry/) looking at the state of machine learning frameworks today. In part, as highlighted by the article, PyTorch is generally more pythonic than alternative frameworks, easier to debug, and is the most-used language in machine learning research by a large and growing margin. While PyTorch's primary alternative, Tensorflow, has attempted to integrate many of PyTorch's features, Tensorflow's implementations come with some inherent limitations highlighted in the article.

Notably, while PyTorch's industry usage has grown, Tensorflow is still (for now) a slight favorite in industry. In practice, the features that make PyTorch attractive for research also make it attractive for education, and the general trend of machine learning research and practice to PyTorch makes it the more proactive choice. 

# Tensor Properties
One way to create tensors from a list or an array is to use `torch.Tensor`. It'll be used to set up examples in this notebook, but you'll never need to use it in the course - in fact, if you find yourself needing it, that's probably not the correct answer. 

In [80]:
example_tensor = torch.Tensor(
    [
     [[1, 2], [3, 4]],
     [[5, 6], [7, 8]],
     [[9, 0], [1, 2]]
    ]
)

You can view the tensor in the notebook by simple printing it out (though some larger tensors will be cut off)

In [81]:
example_tensor

tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]],

        [[9., 0.],
         [1., 2.]]])

## Tensor Properties: Device

One important property is the device of the tensor - throughout this notebook you'll be sticking to tensors which are on the CPU. However, throughout the course you'll also be using tensors on GPU (that is, a graphics card which will be provided for you to use for the course). To view the device of the tensor, all you need to write is `example_tensor.device`. To move a tensor to a new device, you can write `new_tensor = example_tensor.to(device)` where device will be either `cpu` or `cuda`.

In [82]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [83]:
example_tensor = example_tensor.to(device)
example_tensor.device

device(type='cuda', index=0)

In [84]:
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3060 Laptop GPU


## Tensor Properties: Shape

And you can get the number of elements in each dimension by printing out the tensor's shape, using `example_tensor.shape`, something you're likely familiar with if you've used numpy. For example, this tensor is a $3\times2\times2$ tensor, since it has 3 elements, each of which are $2\times2$. 

In [85]:
example_tensor.shape

torch.Size([3, 2, 2])

You can also get the size of a particular dimension $n$ using `example_tensor.shape[n]` or equivalently `example_tensor.size(n)`

In [86]:
print("shape[0] =", example_tensor.shape[0])
print("size(1) =", example_tensor.size(1))

shape[0] = 3
size(1) = 2


Finally, it is sometimes useful to get the number of dimensions (rank) or the number of elements, which you can do as follows

In [87]:
print("Rank =", len(example_tensor.shape))
print("Number of elements =", example_tensor.numel())

Rank = 3
Number of elements = 12


# Indexing Tensors

As with numpy, you can access specific elements or subsets of elements of a tensor. To access the $n$-th element, you can simply write `example_tensor[n]` - as with Python in general, these dimensions are 0-indexed. 

In [88]:
example_tensor[1]

tensor([[5., 6.],
        [7., 8.]], device='cuda:0')

In addition, if you want to access the $j$-th dimension of the $i$-th example, you can write `example_tensor[i, j]`

In [89]:
example_tensor[1, 1, 0]

tensor(7., device='cuda:0')

Note that if you'd like to get a Python scalar value from a tensor, you can use `example_scalar.item()`

In [90]:
example_scalar = example_tensor[1, 1, 0]
example_scalar.item()

7.0

In addition, you can index into the ith element of a column by using `x[:, i]`. For example, if you want the top-left element of each element in `example_tensor`, which is the `0, 0` element of each matrix, you can write:

In [91]:
example_tensor[:, 0, 0]

tensor([1., 5., 9.], device='cuda:0')

# Initializing Tensors

There are many ways to create new tensors in PyTorch, but in this course, the most important ones are: 

[`torch.ones_like`](https://pytorch.org/docs/master/generated/torch.ones_like.html): creates a tensor of all ones with the same shape and device as `example_tensor`.

In [92]:
torch.ones_like(example_tensor)

tensor([[[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]],

        [[1., 1.],
         [1., 1.]]], device='cuda:0')

[`torch.zeros_like`](https://pytorch.org/docs/master/generated/torch.zeros_like.html): creates a tensor of all zeros with the same shape and device as `example_tensor`

In [93]:
torch.zeros_like(example_tensor)

tensor([[[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.]]], device='cuda:0')

[`torch.randn_like`](https://pytorch.org/docs/stable/generated/torch.randn_like.html): creates a tensor with every element sampled from a [Normal (or Gaussian) distribution](https://en.wikipedia.org/wiki/Normal_distribution) with the same shape and device as `example_tensor`


In [94]:
torch.randn_like(example_tensor)

tensor([[[-0.2243, -1.4564],
         [-0.9045, -1.2556]],

        [[ 1.6808, -0.2386],
         [ 0.0474, -2.0658]],

        [[-0.6923, -1.2453],
         [ 0.3211, -1.3030]]], device='cuda:0')

Sometimes (though less often than you'd expect), you might need to initialize a tensor knowing only the shape and device, without a tensor for reference for `ones_like` or `randn_like`. In this case, you can create a $2x2$ tensor as follows:

In [95]:
torch.randn(2, 2, device='cpu') # Alternatively, for a GPU tensor, you'd use device='cuda'

tensor([[-0.0615,  2.2524],
        [ 2.5393,  0.3354]])

# Basic Functions

There are a number of basic functions that you should know to use PyTorch - if you're familiar with numpy, all commonly-used functions exist in PyTorch, usually with the same name. You can perform element-wise multiplication / division by a scalar $c$ by simply writing `c * example_tensor`, and element-wise addition / subtraction by a scalar by writing `example_tensor + c`

Note that most operations are not in-place in PyTorch, which means that they don't change the original variable's data (However, you can reassign the same variable name to the changed data if you'd like, such as `example_tensor = example_tensor + 1`)

In [96]:
(example_tensor - 5) * 2

tensor([[[ -8.,  -6.],
         [ -4.,  -2.]],

        [[  0.,   2.],
         [  4.,   6.]],

        [[  8., -10.],
         [ -8.,  -6.]]], device='cuda:0')

You can calculate the mean or standard deviation of a tensor using [`example_tensor.mean()`](https://pytorch.org/docs/stable/generated/torch.mean.html) or [`example_tensor.std()`](https://pytorch.org/docs/stable/generated/torch.std.html). 

In [97]:
print("Mean:", example_tensor.mean())
print("Stdev:", example_tensor.std())

Mean: tensor(4., device='cuda:0')
Stdev: tensor(2.9848, device='cuda:0')


You might also want to find the mean or standard deviation along a particular dimension. To do this you can simple pass the number corresponding to that dimension to the function. For example, if you want to get the average $2\times2$ matrix of the $3\times2\times2$ `example_tensor` you can write:

In [98]:
example_tensor.mean(0)

# Equivalently, you could also write:
# example_tensor.mean(dim=0)
# example_tensor.mean(axis=0)
# torch.mean(example_tensor, 0)
# torch.mean(example_tensor, dim=0)
# torch.mean(example_tensor, axis=0)

tensor([[5.0000, 2.6667],
        [3.6667, 4.6667]], device='cuda:0')

PyTorch has many other powerful functions but these should be all of PyTorch functions you need for this course outside of its neural network module (`torch.nn`).

# PyTorch Neural Network Module (`torch.nn`)

PyTorch has a lot of powerful classes in its `torch.nn` module (Usually, imported as simply `nn`). These classes allow you to create a new function which transforms a tensor in specific way, often retaining information when called multiple times.

In [99]:
import torch.nn as nn

## `nn.Linear`

To create a linear layer, you need to pass it the number of input dimensions and the number of output dimensions. The linear object initialized as `nn.Linear(10, 2)` will take in a $n\times10$ matrix and return an $n\times2$ matrix, where all $n$ elements have had the same linear transformation performed. For example, you can initialize a linear layer which performs the operation $Ax + b$, where $A$ and $b$ are initialized randomly when you generate the [`nn.Linear()`](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) object. 

In [100]:
linear = nn.Linear(10, 2)
example_input = torch.randn(3, 10)
example_output = linear(example_input)
example_output

tensor([[-1.6089,  0.1074],
        [-0.0125, -0.8623],
        [ 0.2015,  0.6390]], grad_fn=<AddmmBackward0>)

## `nn.ReLU`

[`nn.ReLU()`](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) will create an object that, when receiving a tensor, will perform a ReLU activation function. This will be reviewed further in lecture, but in essence, a ReLU non-linearity sets all negative numbers in a tensor to zero. In general, the simplest neural networks are composed of series of linear transformations, each followed by activation functions. 

In [101]:
relu = nn.ReLU()
relu_output = relu(example_output)
relu_output

tensor([[0.0000, 0.1074],
        [0.0000, 0.0000],
        [0.2015, 0.6390]], grad_fn=<ReluBackward0>)

## `nn.BatchNorm1d`

[`nn.BatchNorm1d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html) is a normalization technique that will rescale a batch of $n$ inputs to have a consistent mean and standard deviation between batches.  

As indicated by the `1d` in its name, this is for situations where you expect a set of inputs, where each of them is a flat list of numbers. In other words, each input is a vector, not a matrix or higher-dimensional tensor. For a set of images, each of which is a higher-dimensional tensor, you'd use [`nn.BatchNorm2d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html), discussed later on this page.

`nn.BatchNorm1d` takes an argument of the number of input dimensions of each object in the batch (the size of each example vector).

In [102]:
batchnorm = nn.BatchNorm1d(2)
batchnorm_output = batchnorm(relu_output)
batchnorm_output

tensor([[-0.7067, -0.5062],
        [-0.7067, -0.8905],
        [ 1.4134,  1.3966]], grad_fn=<NativeBatchNormBackward0>)

## `nn.Sequential`

[`nn.Sequential`](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) creates a single operation that performs a sequence of operations. For example, you can write a neural network layer with a batch normalization as

In [103]:
mlp_layer = nn.Sequential(
    nn.Linear(5, 2),
    nn.BatchNorm1d(2),
    nn.ReLU()
)

test_example = torch.randn(5,5) + 1
print("input: ")
print(test_example)
print("output: ")
print(mlp_layer(test_example))

input: 
tensor([[ 0.8689, -0.4778,  2.6751,  1.5801,  0.9310],
        [ 1.9243,  0.5145,  1.9187, -0.6219,  1.1381],
        [ 1.6162,  4.4616,  0.5915,  1.7859,  0.9952],
        [ 0.4625,  1.3116, -0.8308,  0.5187, -0.0923],
        [ 2.2831,  2.5087, -0.2179,  0.3349, -1.0809]])
output: 
tensor([[0.0000, 1.3717],
        [1.6337, 0.8305],
        [0.0000, 0.0000],
        [0.0000, 0.0000],
        [0.4483, 0.0000]], grad_fn=<ReluBackward0>)


# Optimization

One of the most important aspects of essentially any machine learning framework is its automatic differentiation library. 

## Optimizers

To create an optimizer in PyTorch, you'll need to use the `torch.optim` module, often imported as `optim`. [`optim.Adam`](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam) corresponds to the Adam optimizer. To create an optimizer object, you'll need to pass it the parameters to be optimized and the learning rate, `lr`, as well as any other parameters specific to the optimizer.

For all `nn` objects, you can access their parameters as a list using their `parameters()` method, as follows:

In [114]:
import torch.optim as optim
adam_opt = optim.Adam(linear.parameters(), lr=0.01)

In [111]:
list(mlp_layer.parameters())

[Parameter containing:
 tensor([[ 0.5052, -0.0084,  0.2703, -0.2704, -0.1350],
         [-0.1908, -0.2868,  0.2602, -0.1563, -0.0242]], requires_grad=True),
 Parameter containing:
 tensor([0.4305, 0.0978], requires_grad=True),
 Parameter containing:
 tensor([0.9000, 0.9000], requires_grad=True),
 Parameter containing:
 tensor([0.1000, 0.1000], requires_grad=True)]

### Explanation of MLP Layer Parameters


The `list(mlp_layer.parameters())` output shows 4 parameter tensors from your Sequential model. Here's what each represents:

### Explanation of MLP Layer Parameters

The `list(mlp_layer.parameters())` output shows 4 parameter tensors from your Sequential model. Here's what each represents:

#### 1. Linear Layer Weights
```python
Parameter containing:
 tensor([[ 0.5052, -0.0084,  0.2703, -0.2704, -0.1350],
         [-0.1908, -0.2868,  0.2602, -0.1563, -0.0242]], requires_grad=True)
```
- **Shape**: [2, 5] - corresponds to the output and input dimensions of `nn.Linear(5, 2)`
- **Role**: Weight matrix (W) used in the transformation y = Wx + b
- **Each row**: Represents weights for one output neuron

#### 2. Linear Layer Bias
```python
Parameter containing:
 tensor([0.4305, 0.0978], requires_grad=True)
```
- **Shape**: [2] - matches the output dimension of the linear layer
- **Role**: Bias vector (b) added after matrix multiplication
- **Values**: Added to each output before passing to the next layer

#### 3. BatchNorm1d Running Mean
```python
Parameter containing:
 tensor([0.9000, 0.9000], requires_grad=True)
```
- **Shape**: [2] - matches the dimension of features being normalized
- **Role**: Learnable scale factor (gamma) for BatchNorm1d
- **Function**: Scales the normalized values (default initialization is often 1.0)
- **Note**: These appear to be initialized to 0.9 rather than 1.0

#### 4. BatchNorm1d Running Variance
```python
Parameter containing:
 tensor([0.1000, 0.1000], requires_grad=True)
```
- **Shape**: [2] - matches the number of features
- **Role**: Learnable shift parameter (beta) for BatchNorm1d
- **Function**: Shifts the normalized values (default initialization is usually 0)
- **Note**: These appear to be initialized to 0.1 rather than 0.0

Note that BatchNorm1d also maintains running statistics (mean and variance) during training, but these aren't considered parameters because they don't receive gradient updates - they're updated via a moving average.


## Training Loop

A (basic) training step in PyTorch consists of four basic parts:


1.   Set all of the gradients to zero using `opt.zero_grad()`
2.   Calculate the loss, `loss`
3.   Calculate the gradients with respect to the loss using `loss.backward()`
4.   Update the parameters being optimized using `opt.step()`

That might look like the following code (and you'll notice that if you run it several times, the loss goes down):


In [105]:
train_example = torch.randn(100,5) + 1
adam_opt.zero_grad()

# We'll use a simple loss function of mean distance from 1
# torch.abs takes the absolute value of a tensor
cur_loss = torch.abs(1 - mlp_layer(train_example)).mean()

cur_loss.backward()
adam_opt.step()
print(cur_loss)

tensor(0.7644, grad_fn=<MeanBackward0>)


## `requires_grad_()`

You can also tell PyTorch that it needs to calculate the gradient with respect to a tensor that you created by saying `example_tensor.requires_grad_()`, which will change it in-place. This means that even if PyTorch wouldn't normally store a grad for that particular tensor, it will for that specified tensor. 

In [121]:
# Create a tensor without gradient tracking
x = torch.tensor([2.0, 3.0])
print(f"Initially requires_grad: {x.requires_grad}")  # False

# Enable gradient tracking in-place
x.requires_grad_()
print(f"After requires_grad_: {x.requires_grad}")  # True

# Now operations are tracked
y = x * x
z = y.sum()

# We can calculate gradients
z.backward()
print(f"Gradient: {x.grad}")  # tensor([4., 6.])
print(f"z: {z}")  # tensor(13., grad_fn=<SumBackward0>)

Initially requires_grad: False
After requires_grad_: True
Gradient: tensor([4., 6.])
z: 13.0


## `with torch.no_grad():`

PyTorch will usually calculate the gradients as it proceeds through a set of operations on tensors. This can often take up unnecessary computations and memory, especially if you're performing an evaluation. However, you can wrap a piece of code with `with torch.no_grad()` to prevent the gradients from being calculated in a piece of code. 

- Training: requires_grad=True → Enables gradient calculation
- Evaluation: requires_grad=False → Disables gradient calculation

```python
with torch.no_grad():  # Important for efficiency
    for inputs, targets in val_loader:
        outputs = model(inputs)
        # ... evaluation code ...
```


## `detach():`

Sometimes, you want to calculate and use a tensor's value without calculating its gradients. For example, if you have two models, A and B, and you want to directly optimize the parameters of A with respect to the output of B, without calculating the gradients through B, then you could feed the detached output of B to A. There are many reasons you might want to do this, including efficiency or cyclical dependencies (i.e. A depends on B depends on A).

## What `detach()` Does

`detach()` creates a new tensor that shares the same data as the original tensor but doesn't require gradients or track the computational history:

```python
detached_tensor = original_tensor.detach()
```

## Key Characteristics

1. **Breaks Gradient Flow**:
   - Creates a "checkpoint" in your computational graph
   - Gradients won't flow backward beyond this point
   - Original tensor remains unchanged

2. **Shares Underlying Data**:
   - Both tensors point to the same memory location
   - Changes to the underlying data will be reflected in both tensors
   - Unlike `requires_grad=False` which only disables gradient tracking

## Common Use Cases

### 1. Model A Depending on Model B's Output

```python
# Forward pass through model B
output_B = model_B(input_data)

# Detach output of B before feeding to model A
detached_output = output_B.detach()

# Forward pass through model A
output_A = model_A(detached_output)

# Only model A's parameters will be updated
loss = loss_function(output_A, target)
loss.backward()
optimizer_A.step()
```

### 2. Implementation Examples

#### GAN Training
```python
# Training discriminator - don't update generator
fake_images = generator(noise).detach()  # No gradients to generator
real_loss = discriminator(real_images)
fake_loss = discriminator(fake_images)
```

#### Reinforcement Learning
```python
# Fixed target networks for stable learning
next_q_values = target_network(next_states).detach()
```

#### Avoiding Cyclical Dependencies
```python
# When A depends on B and B depends on A
intermediate = model_A(input_data)
output_B = model_B(intermediate.detach())
final_output = model_A(output_B)
```

## Difference from `with torch.no_grad()`

- `detach()` creates a new tensor without gradient history
- `with torch.no_grad():` temporarily disables gradient calculation globally

This feature is crucial for advanced training procedures like GANs, actor-critic methods, and other scenarios where selective gradient flow is needed.


# New `nn` Classes

You can also create new classes which extend the `nn` module. For these classes, all class attributes, as in `self.layer` or `self.param` will automatically treated as parameters if they are themselves `nn` objects or if they are tensors wrapped in `nn.Parameter` which are initialized with the class. 

The `__init__` function defines what will happen when the object is created. The first line of the init function of a class, for example, `WellNamedClass`, needs to be `super(WellNamedClass, self).__init__()`. 

The `forward` function defines what runs if you create that object `model` and pass it a tensor `x`, as in `model(x)`. If you choose the function signature, `(self, x)`, then each call of the forward function, gets two pieces of information: `self`, which is a reference to the object with which you can access all of its parameters, and `x`, which is the current tensor for which you'd like to return `y`.

One class might look like the following:

In [ ]:
class ExampleModule(nn.Module):
    def __init__(self, input_dims, output_dims):
        super(ExampleModule, self).__init__()
        self.linear = nn.Linear(input_dims, output_dims)
        # self.exponent = nn.Parameter(torch.tensor(1.))  # no change here

    def forward(self, x):
        x = self.linear(x)

        # This is the notation for element-wise exponentiation,
        # which matches python in general
        # x = x ** self.exponent   # doesn't change x

        return x

And you can view its parameters as follows

In [107]:
example_model = ExampleModule(10, 2)
list(example_model.parameters())

[Parameter containing:
 tensor(1., requires_grad=True),
 Parameter containing:
 tensor([[-0.0860, -0.2406,  0.2332, -0.0101, -0.2965,  0.1594, -0.0803, -0.3149,
          -0.2815, -0.1120],
         [-0.1636, -0.2184, -0.0665,  0.2424, -0.1925, -0.2150, -0.2374, -0.0985,
          -0.2911,  0.0387]], requires_grad=True),
 Parameter containing:
 tensor([0.2491, 0.1130], requires_grad=True)]

And you can print out their names too, as follows:

In [108]:
list(example_model.named_parameters())

[('exponent',
  Parameter containing:
  tensor(1., requires_grad=True)),
 ('linear.weight',
  Parameter containing:
  tensor([[-0.0860, -0.2406,  0.2332, -0.0101, -0.2965,  0.1594, -0.0803, -0.3149,
           -0.2815, -0.1120],
          [-0.1636, -0.2184, -0.0665,  0.2424, -0.1925, -0.2150, -0.2374, -0.0985,
           -0.2911,  0.0387]], requires_grad=True)),
 ('linear.bias',
  Parameter containing:
  tensor([0.2491, 0.1130], requires_grad=True))]

And here's an example of the class in action:

In [134]:
input = torch.randn(2, 10)
example_model(input)

tensor([[0.3700, 0.2282],
        [0.9832, 0.1797]], grad_fn=<PowBackward1>)

In [147]:
input2 = torch.ones_like(input)
example_model(input2)

tensor([[-0.7803, -1.0889],
        [-0.7803, -1.0889]], grad_fn=<PowBackward1>)

# 2D Operations

You won't need these for the first lesson, and the theory behind each of these will be reviewed more in later lectures, but here is a quick reference: 


*   2D convolutions: [`nn.Conv2d`](https://pytorch.org/docs/master/generated/torch.nn.Conv2d.html) requires the number of input and output channels, as well as the kernel size.
*   2D transposed convolutions (aka deconvolutions): [`nn.ConvTranspose2d`](https://pytorch.org/docs/master/generated/torch.nn.ConvTranspose2d.html) also requires the number of input and output channels, as well as the kernel size
*   2D batch normalization: [`nn.BatchNorm2d`](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) requires the number of input dimensions
*   Resizing images: [`nn.Upsample`](https://pytorch.org/docs/master/generated/torch.nn.Upsample.html) requires the final size or a scale factor. Alternatively, [`nn.functional.interpolate`](https://pytorch.org/docs/stable/nn.functional.html#torch.nn.functional.interpolate) takes the same arguments. 



